# This notebook details the cleaning process for the Cyclistic case study, the Capstone from the Google Analytics Professional Certificate.
The entire presentation is available [here](https://drive.google.com/file/d/1TJVFIrYrOVh_mlhPSfGv1TTDRzKcJc2q/view?usp=sharing)

In [ ]:
#importing the necessary libraries for the cleaning and analysis process
library("tidyverse")
library("lubridate")
library("skimr")

In [ ]:
#importing the csv files containing the trip data. The data consists of four quarters in separate files. They will be combined later in the process
trips_q2_19 <- read_csv("/kaggle/input/cyclistic/Divvy_Trips_2019_Q2.xlsx.csv")
trips_q3_19 <- read_csv("/kaggle/input/cyclistic1/Divvy_Trips_2019_Q3.csv")
trips_q4_19 <- read_csv("/kaggle/input/cyclistic1/Divvy_Trips_2019_Q4.csv")
trips_q1_20 <- read_csv("/kaggle/input/cyclistic1/Divvy_Trips_2020_Q1.csv")

In [ ]:
#checking the column names to ensure they match so that all the data can be combined in one place
colnames(trips_q2_19)
colnames(trips_q3_19)
colnames(trips_q4_19)
colnames(trips_q1_20)

We see that we have the same columns from q2 2019 to Q4 2019. But the names are different. Some new columns were also added in Q1 2020 while others were removed. Hence, analysis will be based on the common columns

In [ ]:
#first we edit the names of the columns on q2 19 so they are the same names as those in q3 and q4.
trips_q2_19 <- rename(trips_q2_19, 
                      "trip_id" = "01 - Rental Details Rental ID", 
                      "start_time" = "01 - Rental Details Local Start Time", 
                      "end_time" = "01 - Rental Details Local End Time", 
                      "bikeid" = "01 - Rental Details Bike ID", 
                      "tripduration" = "01 - Rental Details Duration In Seconds Uncapped", 
                      "from_station_id" = "03 - Rental Start Station ID", 
                      "from_station_name" = "03 - Rental Start Station Name", 
                      "to_station_id" = "02 - Rental End Station ID", 
                      "to_station_name" = "02 - Rental End Station Name", 
                      "usertype" = "User Type", 
                      "gender" = "Member Gender", 
                      "birthyear" = "05 - Member Details Member Birthday Year")

In [ ]:
#to combine 2019 with 2020 data, we need the column names and types to match
#fixing the different datetime types/formats
trips_q2_19$start_time = dmy_hms(trips_q2_19$start_time)
trips_q2_19$end_time = dmy_hms(trips_q2_19$end_time)
trips_q3_19$start_time = ymd_hms(trips_q3_19$start_time)
trips_q3_19$end_time = ymd_hms(trips_q3_19$end_time)
trips_q4_19$start_time = ymd_hms(trips_q4_19$start_time)
trips_q4_19$end_time = ymd_hms(trips_q4_19$end_time)
trips_q1_20$started_at = ymd_hms(trips_q1_20$started_at)
trips_q1_20$ended_at = ymd_hms(trips_q1_20$ended_at)

In [ ]:
#combining the three 2019 quarters in the same dataframe then reconcile column names with 2020 names
trips_2019 <- rbind(trips_q2_19, trips_q3_19, trips_q4_19)

trips_2019 <- rename(trips_2019,
                    "start_station_name" = "from_station_name",
                    "start_station_id" = "from_station_id",
                    "end_station_name" = "to_station_name",
                    "end_station_id" = "to_station_id",
                    )
trips_q1_20 <- rename(trips_q1_20,
                     "trip_id" = "ride_id",
                     "start_time" = "started_at",
                     "end_time" = "ended_at",
                     "usertype" = "member_casual")

In [ ]:
#q1 2020 data doesn't have a tripduration column.
#creating a new trip duration column for q1 2020 and updating the one for 2019 so the data types match
trips_q1_20$tripduration <- as.numeric(trips_q1_20$end_time - trips_q1_20$start_time)
trips_2019$tripduration <- as.numeric(trips_2019$end_time - trips_2019$start_time)

In [ ]:
#stacking the two dataframes into one keeping only the common columns
all_trips = rbind(
    select(trips_2019, 'trip_id', 'start_time', 'end_time', 'tripduration', 'start_station_id', 'start_station_name','end_station_id','end_station_name','usertype'),
    select(trips_q1_20, 'trip_id', 'start_time', 'end_time', 'tripduration', 'start_station_id', 'start_station_name','end_station_id','end_station_name','usertype'))
str(all_trips)

In [ ]:
#inspecting the dataframe columns for quality issues
skim_without_charts(all_trips)

**The above shows the following issues with the data**
1. One null value in end_station_id and end_station_name: this will be removed
2. Negative trip duration: A trip cannot start earlier than it ended. these rows will be removed
3. Some trips are longer than a day. Since the business doesn't allow a bike to be rented for more than one day, all trips above 1440 minutes are bad data and will be removed. The current trip duration are in seconds and will be converted to minutes
4. There are four member types instead of two. The names of the user segments were changed. These will be reconciled.

In [ ]:
#removing the missing values in end station
alltrips <- all_trips[is.na(all_trips$end_station_name)==FALSE,]
#note that the name of the dataframe here was changed to preserve the original combined dataframes before removing rows. This forms a sort of checkpoint so that if some rows are mistakenly removed, the analyst doesn't have to return to square 1 to fix it.

In [ ]:
#convert tripduration to minutes
alltrips <- alltrips %>% mutate(tripduration = tripduration/60)
#remove the following:
#1. all trip duration below 0 and above 1440
#2. trips starting at 'HQ QR' are not actually trips but are bikes removed from their docking station for quality control
alltrips <- alltrips[!c(alltrips$tripduration<0
                        |alltrips$tripduration>1440
                        |alltrips$start_station_name == 'HQ QR'),]

In [ ]:
#reconcile old and new names for user groups
alltrips$usertype <- alltrips$usertype %>%
str_replace( "Subscriber", "member") %>%
str_replace("Customer", "casual")

In [ ]:
#check that the cleaning was successful
skim_without_charts(alltrips)

In [ ]:
#there are many outliers to the right 
alltrips %>% select(tripduration)%>% ggplot(aes(x=tripduration))+geom_boxplot()
#the boxplot looks pretty strange. While there seem to be many outliers, they are so many that we can't chalk them down to measurement errors. It simply means there are some users that use the bikes very intensively beyond the average

In [ ]:
#investigating outliers
q1 = quantile(alltrips$tripduration, probs=0.25)
q3 = quantile(alltrips$tripduration, probs=0.75)
IQR = q3-q1
lb = q1 - (1.5*IQR)
ub = q3 + (1.5*IQR)
outliers <- alltrips[alltrips$tripduration<lb|alltrips$tripduration>ub,]
skim_without_charts(outliers)

The investigation of the outliers shows that over 200k rows will be considered as outliers looking at this.
Also, all of the "outliers" fall within reasonable range of a bicycle ride (it's not too strange to ride a bicycle for several hours). And this seems like a reasonable frequent occurrence. Hence, the outliers will be left as is since they can't be written off as bad data

In [ ]:
#taking a look at the distribution ride lengths. It seems reasonable that more people will make shorter trips than longer
alltrips %>% ggplot(aes(x=tripduration))+geom_histogram()

In [ ]:
#a closer look at the long tail shows a bump at the end, which indicates that some users rent the bikes for an entire day
alltrips %>% ggplot(aes(x=tripduration))+geom_histogram()+coord_cartesian(ylim=c(0,100000))

# Analysis

**Question: How do annual members and casual users differ?**

In [ ]:
#creating new columns to perform time based calculations
alltrips <- alltrips%>%mutate(
    ride_month = month(start_time, label=TRUE), 
    weekday = wday(start_time,label=TRUE), 
    date = mday(start_time), 
    timeofday = hour(start_time)
)

In [ ]:
#how many trips are started by casual vs members respectively?
alltrips%>%group_by(usertype) %>% summarise(number_of_rides = n(),) %>%mutate(perc_total = number_of_rides/sum(number_of_rides))

members are responsible for majority of trips started

In [ ]:
#what is the average ride length for each user group?
aggregate(alltrips$tripduration ~ alltrips$usertype, FUN = mean)
aggregate(alltrips$tripduration ~ alltrips$usertype, FUN = median)

the average casual trip is three times the length of the member trip. This means they are spending a lot of time riding but they prefer to do this without subscribing.

In [ ]:
#number of trips by day of week
alltrips%>%group_by(usertype,weekday)%>%
mutate(number_of_rides=n())%>%
ggplot(aes(x=alltrips$weekday, y=number_of_rides, color=usertype))+
geom_point()

members cycle mostly on weekdays (explains the shorter trip lengths) while casual users have more trips on weekends. we should encourage casual to ride to work.

In [ ]:
#number of trips by time of day
alltrips%>%group_by(usertype,timeofday)%>%
mutate(number_of_rides=n())%>%
ggplot(aes(x=alltrips$timeofday, y=number_of_rides, color=usertype))+
geom_point()
#spike around 7pm for members and even casual though not as high. Strengthens the initial hypothesis. Let's see if this spike exists for both weekend and weekday

In [ ]:
#number of trips by time of day during the weekend. The 7pm outlier persists but the morning spike is gone
weekend <- c('Sat', 'Sun')
alltrips[alltrips$weekday==weekend,]%>%group_by(usertype, timeofday)%>%mutate(number_of_rides=n())%>%ggplot(aes(x=timeofday, y=number_of_rides, color=usertype))+geom_point()

In [ ]:
#number of trips by time of day during weekdays
alltrips[alltrips$weekday %in% weekend==FALSE,]%>%
group_by(usertype,timeofday)%>%
mutate(number_of_rides=n())%>%
ggplot(aes(x=timeofday, y=number_of_rides, color=usertype))+
geom_point()
#perhaps casual users use bikes as a last resort at closing time even if they didn't take it to work

In [ ]:
#month by month trend
alltrips%>%group_by(usertype, ride_month)%>%mutate(number_of_rides=n())%>%ggplot(aes(x=ride_month, y=number_of_rides, color=usertype))+geom_point()

Number of trips trend by month, with more trips during the summer months. However, the trends are similar for both casual users and members. Hence, this doesn't offer insights to how the marketing strategy for members can be differentiated from those for casual users

In [ ]:
#what stations do trips most commonly start from?
station_trips <- aggregate(alltrips$trip_id ~ alltrips$usertype + alltrips$start_station_name+alltrips$weekday, FUN = length)%>% arrange(desc("trip_id"))%>%pivot_wider(names_from = "alltrips$usertype", values_from = "alltrips$trip_id")

In [ ]:
colnames(station_trips)

In [ ]:
station_trips <- rename(station_trips,
                        'start_station' = 'alltrips$start_station_name', 
                        'weekday' = 'alltrips$weekday'
                       )

In [ ]:
#where do members start trips from the most? 
station_trips%>%group_by(start_station)%>%summarise(n_trips = sum(member))%>%arrange(desc(n_trips))

In [ ]:
#where do casual users start trips from the most? 
station_trips%>%group_by(start_station)%>%summarise(n_trips = sum(casual))%>%arrange(desc(n_trips))

This suggests there is a geographical divide between members and casual users

In [ ]:
#create a summarized data frame with number of rides and mean ride duration by usertype, weekday and time of day
n_ride <- aggregate(alltrips$trip_id ~ alltrips$usertype +date(alltrips$start_time)+ alltrips$weekday +alltrips$timeofday, FUN = length)
mean_length <- aggregate(alltrips$tripduration ~ alltrips$usertype +date(alltrips$start_time)+alltrips$weekday +alltrips$timeofday , FUN = sum)
df_export <- merge(n_ride,mean_length, sort = FALSE)

In [ ]:
setwd('/kaggle/working')

write.csv(df_export, "summarized_cyclistic.csv")
write.csv(station_trips, "stations_start.csv")

In [ ]:
list.files(path = "../working")